# Installing Libraries

In [ ]:
!pip install datasets transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset

# Load our data
data = load_dataset("rotten_tomatoes")
data

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [ ]:
data["train"][0]

{'text': 'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .',
 'label': 1}

In [ ]:
data["train"][7000]

{'text': "it won't be long before you'll spy i spy at a video store near you .",
 'label': 0}

# Text Classification using Generative Models

In [ ]:
from transformers import pipeline

# Load our model
model_id = "google/flan-t5-small"
pipe = pipeline(
    "text2text-generation",
    model= model_id,
    device="cuda:0"
)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cuda:0


# Lets extract Test data

In [ ]:
test_data = data['test']

# Lets play with Flan-T5

In [ ]:
prompt = "What is sentiment of input sentence?"
sample_test_datapoint = test_data['text'][0]
print(sample_test_datapoint)

final_input = prompt + sample_test_datapoint

prediction = pipe(final_input)
print(prediction)

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .
[{'generated_text': 'positive'}]


# A more perfect prompt

In [ ]:
prompt = "Is the sentiment of input movie review 1 or 0?"
sample_test_datapoint = test_data['text'][12]
print(sample_test_datapoint)

final_input = prompt + sample_test_datapoint

prediction = pipe(final_input)
print(prediction)

mostly , [goldbacher] just lets her complicated characters be unruly , confusing and , through it all , human .
[{'generated_text': '0'}]


 # Lets do evaluation on entire test set

In [ ]:
from tqdm import tqdm

prompt = "Is the sentiment of input movie review positive or negative?"

predictions = []

for each_test_datapoint in tqdm(test_data['text']):
  test_sentence = prompt + each_test_datapoint
  prediction = pipe(test_sentence)[0].get('generated_text')
  if prediction == "positive":
    predictions.append(1)
  elif prediction == "negative":
    predictions.append(0)
  else:
    print('ALERT: ', prediction)

100%|██████████| 1066/1066 [00:44<00:00, 23.88it/s]


In [ ]:
from sklearn.metrics import classification_report

y_true = data['test']["label"]

report = classification_report(y_true, predictions,
                               target_names=["Negative Movie Reiew", "Positive Movie Review"])
print(report)

                       precision    recall  f1-score   support

 Negative Movie Reiew       0.84      0.86      0.85       533
Positive Movie Review       0.86      0.84      0.85       533

             accuracy                           0.85      1066
            macro avg       0.85      0.85      0.85      1066
         weighted avg       0.85      0.85      0.85      1066

